In [4]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import AutoTokenizer, AutoModelWithLMHead
import torch
import re
from tqdm import tqdm
import os

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
variant = "gpt2"
out_dir = f"outputs/{variant}"
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

In [7]:
tokenizer = AutoTokenizer.from_pretrained(variant)
model = AutoModelWithLMHead.from_pretrained(variant).to(device)

/home/bishal/miniconda3/lib/python3.7/site-packages/transformers/modeling_auto.py:837: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [8]:
stack = []
with open('daily_dialog/test/dialogues_test.txt') as f, open(f'{out_dir}/dialgpt_responses.txt', 'w') as of:
    lines = f.readlines()
    
    for i, line in tqdm(enumerate(lines), total=len(lines)):
        dialog = re.split(" *__eou__ *", line.strip())
        
        predictions = {
            "filename": i,
            "utterances": []
        }
        context = []
        
        dialog = dialog[:-1] # last one is empty
        for step, u in enumerate(dialog):
            if step == 0:
                predictions['utterances'].append(
                    {
                        "true": u
                    }
                )
            else:
                predictions['utterances'][-1]['true'] = u
                
            # No further utterances
            if step == len(dialog) - 1:
                break
                
            # print("\nuser:", u)
            context.append(u)
            # encode the new user input, add the eos_token and return a tensor in Pytorch
            # new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt').to(device)
            new_user_input_ids = tokenizer.encode(u, return_tensors='pt').to(device)

            # append the new user input tokens to the chat history
            # bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids
            bot_input_ids = torch.cat([bot_input_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

            # generated a response while limiting the total chat history to 1000 tokens, 
            chat_history_ids = model.generate(bot_input_ids, max_length=bot_input_ids.size(1) + 30, pad_token_id=tokenizer.eos_token_id)

            # pretty print last ouput tokens from bot
            resp = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
            
            for k, s in enumerate(context):
                of.write(f"Turn {k}: {s}\n")
            of.write(f"GPT2: {resp}\n\n")
            
            predictions['utterances'].append(
                    {
                        "predicted": resp
                    }
                )
        stack.append(predictions)

100%|██████████| 1000/1000 [50:43<00:00,  3.04s/it] 


In [10]:
import json
with open(f"{out_dir}/test_predictions.json", "w") as f:
    json.dump(stack, f)

In [ ]:
# with open('daily_dialog/test/dialogues_test.txt') as f, open('outputs/gpt2_responses.txt', 'w') as of:
#     lines = f.readlines()
#     for line in tqdm(lines):
#         dialog = re.split(" *__eou__ *", line.strip())
        
#         context = []
#         for step, u in enumerate(dialog):
#             if len(u) < 1:
#                 continue
#             # print("\nuser:", u)
#             u = u + " "
#             context.append(u)
#             # encode the new user input, add the eos_token and return a tensor in Pytorch
#             # new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt').to(device)
#             new_user_input_ids = tokenizer.encode(u, return_tensors='pt').to(device)

#             # append the new user input tokens to the chat history
#             # bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids
#             bot_input_ids = torch.cat([bot_input_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

#             # generated a response while limiting the total chat history to 1000 tokens, 
#             chat_history_ids = model.generate(bot_input_ids, max_length=bot_input_ids.size(1) + 30, pad_token_id=tokenizer.eos_token_id)

#             # pretty print last ouput tokens from bot
#             resp = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
            
            
#             for k, s in enumerate(context):
#                 of.write(f"Turn {k}: {s}\n")
#             of.write(f"GPT2: {resp}\n\n")